# Crafting a Personalized Morning Update with News and Weather APIs

## 🎯 Project Goal  
Build a fully automated and personalized **morning audio briefing** using external APIs and OpenAI services. We'll combine **weather updates**, **top news headlines**, and **natural language generation** into a seamless, voice-enabled experience.

---

## 🧠 What You’ll Learn

- How to **interact with external APIs** (NewsAPI & Meteosource) using Python
- How to use the **OpenAI Chat Completions API** to summarize and format data
- How to generate **natural-sounding audio** with OpenAI’s Text-to-Speech (TTS)
- How to build a **workflow** that combines data extraction, transformation, and delivery in an engaging format

---

## 🚀 Project Overview

Welcome to this exciting project where **Python meets the real world**. Imagine starting your day with a **custom audio summary** that tells you:

- 🌦️ The latest weather forecast for your location  
- 📰 The top 10 news headlines tailored for your interests  
- 🗣️ All delivered in a natural, conversational tone — straight to your ears

We’ll fetch live data from:
- **🔹 Meteosource** – for localized and accurate weather information  
- **🔹 NewsAPI** – to retrieve breaking news from trusted sources  

Then, we’ll pass this data to:
- **🧠 OpenAI Chat Completions API** – to craft a personalized script  
- **🔊 OpenAI Text-to-Speech API** – to produce a downloadable MP3 file  

---

## 💡 Why This Matters

This project brings together several real-world skills:
- API integration
- Data wrangling and formatting
- Generative AI for content creation
- Audio synthesis
- Workflow automation

Whether you're a data scientist, developer, or AI enthusiast, this project demonstrates how to **orchestrate multiple technologies** into a meaningful, personalized user experience.

---

## 🧭 Let’s Get Started

Grab your coffee and your keyboard — it’s time to code your way into a **smarter morning routine**!


## Before We Begin: Preparation

For this project, we’ll be using several APIs available online.

- [Meteosource](https://www.meteosource.com/): The free plan is sufficient for our needs.
- [NewsAPI](https://newsapi.org/): The free plan is sufficient for our needs.
- [OpenAI API](https://platform.openai.com/): Requires a paid plan.

This will allow us to seamlessly integrate weather, news, and AI capabilities into our personalized morning update.

## A quick refresher on API concepts

APIs (Application Programming Interfaces) enable different systems to communicate and share data seamlessly, forming the backbone of many digital interactions. REST APIs, a common type, use HTTP to let systems communicate over the internet through a request-response cycle. Key HTTP verbs include `GET` (read a resource), `POST` (create a new resource), `PUT` (update an existing resource), and `DELETE` (delete a resource). REST APIs are accessed via URLs, which include the protocol, domain, port, path, and optional query parameters. Headers provide additional context for requests and responses, such as authentication and content type.

HTTP status codes indicate the result of a request and are grouped into five categories: `1XX` (Informational), `2XX` (Success), `3XX` (Redirection), `4XX` (Client errors), and `5XX` (Server errors). Key codes include `200` (OK), `404` (Not Found), and `500` (Internal Server Error). Understanding these concepts will help as navigate and utilize APIs effectively in our project here.

## Preparing our _Enviroment_

We will be using the `os` and `requests` packages. 


### Storing secret API´s

All three of the APIs we plan on using support API-key based authentication so let's set up these API keys as environment variables for our workbook. We should not copy-paste these API-keys directly into our python code because that risks accidentally leaking them when you for example would copy the workbook. Given some of these APIs incur a cost when using them, it's of utmost importance we store them securely.

### Steps we follow for this:

1. Create three environment variables with the following names
    - `API_KEY_METEOSOURCE`
    - `API_KEY_NEWSAPI`
    - `API_KEY_OPENAI`
2. Save the environment variables with a name of your choice
3. Connect the environment variables to our Notebook
4. Load all API keys into Python variables using the `os` package

In [2]:
# Import the os package
import os

# Load the environment variables we've set up into Python variables
API_KEY_METEOSOURCE = os.environ["API_KEY_METEOSOURCE"]
API_KEY_NEWSAPI = os.environ["API_KEY_NEWSAPI"]
API_KEY_OPENAI = os.environ["OPENAI_API_KEY"]

## Getting the latest news headlines

### Objectives

In this task, we'll use the NewsAPI (newsapi.org) to fetch the most recent news headlines. We aim to include these headlines in an OpenAI prompt later. Therefore, we'll need to retrieve both the title and a short description for each news item.

#### Explanation

1. **NewsAPI**: This is a service that provides access to news articles from various sources. We'll use it to get the latest headlines.
2. **OpenAI Prompt**: Later, we'll use the fetched news headlines as part of a prompt for OpenAI's language model.
3. **Data Requirements**: For each news item, we need to extract:
   - **Title**: The headline of the news article.
   - **Description**: A brief summary or description of the news article.

- Import the `requests` library.

In [3]:
import requests
from IPython.display import display, Markdown

Now let's consult the [newsapi.org documentation](https://newsapi.org/docs/). We'll be using the _top headlines_ API endpoint. This API has a few required URL parameters, which means we'll need to include these in our API call, otherwise we'll get an error!

The newsapi.org API also requires authentication by sending along an API-key in every request we make. The documentation shows us that we can send this API key as a URL parameter called `apiKey` (note the capitalization!).

- We will create a new Python dictionary called `newsapi_url_parameters` with all the required URL parameters for our API request.

In [4]:
newsapi_url_parameters = {
    "apikey": API_KEY_NEWSAPI,
    "country": "us",
    "pageSize": 10
}

Now we've got everything ready, let's fire the API request and save the data we need to a variable we'll use later.

- Send the API request to the `/top-headlines` API.
- Evaluate if the response is successful.
- Create a new list called `headline_articles`
- Add dictionaries with the `title` and `description` for each article to the list.

In [5]:
# GET requests
response = requests.get(
    "https://www.newsapi.org/v2/top-headlines", 
    params = newsapi_url_parameters
)
# status_code evaluation
if (response.status_code == 200):
    response_json = response.json()
    headline_articles = [
        {'title': article['title'], 'description': article['description']} for article in response_json['articles'] 
    ]
    print(headline_articles)

# Raise a SystemExit exception if status code does not work
else:
    raise SystemExit(f'An error occured fetching the news headlines ({response.status_code}, {response.text})')

[{'title': 'Ananda Lewis, former MTV VJ who shared breast cancer journey, has died at 52 - CNN', 'description': 'Ananda Lewis, a former MTV VJ who connected a generation of music fans to their favorite celebrities in the network’s heyday, has died. She was 52.'}, {'title': 'Trump speech prompts concerns about politicization of military - NBC News', 'description': 'Uniformed soldiers cheered and booed along with political statements, something typically frowned upon and potentially a violation of military regulations.'}, {'title': 'Hogg declines to run again for DNC vice chair after new election is called - Politico', 'description': 'The 25-year-old activist said he will devote his efforts to Leaders We Deserve.'}, {'title': "New species of dinosaur discovered that 'rewrites' T.rex family tree - BBC", 'description': 'The dinosaur skeletons, found hidden in a museum collection in Mongolia, is an ancestor of the mighty tyrannosaurs.'}, {'title': 'Trump White House opens door to historic m

## Fetch the Eeather Forecast from Meteosource API

Now let's fetch the weather forecast. We'll use the `requests` library to perform an API request to the meteosource.com API. Reading the 
meteosource.com API documentation we learn that we need to use the /point "weather and forecast" API to obtain a forecast for a specific geographic location.

The documentation also notes that any API request requires a place_id for the location, so we must find that first. Fortunately, meteosource offers an API that allows us to easily search for the place_id of any location: /find_places.

The MeteoSource API also requires authentication, this time we'll use the headers to pass on the API key, we'll need to use the x-api-key header, so let's first start by creating the dictionary of headers.

- We can create a new dictionary named meteosource_headers with the correct authentication key and value.

In [6]:
meteosource_headers = {"X-API-Key": API_KEY_METEOSOURCE}

Cool, we've got the headers in place! Now we need to create a dictionary of URL parameters. The `/find_places` API just requires a single URL parameter: `text`. 

- Create a dictionary named `meteosource_findplaces_url_parameters` with one key-value pair that contains the location you want to find the `place_id` for.

In [7]:
meteosource_findplaces_url_parameters = {"text": "Buenos Aires City"}

Done! We now have both the headers and URL parameters ready to pass along with the API request. Let's send the request and evaluate the response. We'll extract the place_id of the first match in the response and store it in the `place_id` variable so we can use it later.

- Send the API request to the `/find_places` API.
- Evaluate if the response is successful.
- Extract the first place_id from the response and store it in the variable `place_id`.

In [8]:
response = requests.get(
	"https://www.meteosource.com/api/v1/free/find_places", 
  params=meteosource_findplaces_url_parameters,
  headers=meteosource_headers
 )

if (response.status_code == 200):
    response_json = response.json()
    
    if len(response_json) > 0:
        place_id = response_json[0]['place_id']
        print(place_id)
    else:
        raise SystemExit('Could not find any place_id for the location')
    
else:
    raise SystemExit(f'An error occured fetching the place_id ({response.status_code}, {response.text})')

city-bell-3435379


Now we've got the `place_id`, we can move on and fetch the actual weather forecast for our location.

We can re-use the `meteosource_headers` variable from before for authentation, but we'll need to create a new dictionary with URL parameters. Reading the [meteosource.com `/point` API documentation](https://www.meteosource.com/documentation#point) to learn which URL parameters are needed to fetch the daily forecast we need for our update message.

- Create a dictionary named `meteosource_point_url_parameters` with all of the required URL variables for the `/point` API.

In [9]:
meteosource_point_url_parameters = {
    "place_id": place_id,
    "sections": "daily",
    "units": "metric"
}

Now we're ready to fire the request to the MeteoSource point API, and get our daily forecast. Check the documentation to learn more about the exact response format, so we can also extract today's forecast and store it in a variable.

- Send the API request to the `/point` API.
- Evaluate if the response is successful.
- Check if the response contains a daily forecast
- Store the whole daily forecast object in the `forecast_data` variable, only do this for today's / hourly's / daily´s, etc. forecast

In [10]:
response = requests.get("https://www.meteosource.com/api/v1/free/point", 
                        params=meteosource_point_url_parameters,
                        headers=meteosource_headers)

if (response.status_code == 200):
    response_json = response.json()
    
    if len(response_json['daily']['data']) > 0:
        print(f"The Weather in {meteosource_findplaces_url_parameters['text']} is:")
        forecast_summary = response.json()['daily']['data'][0]['summary']
    else:
        raise SystemExit('Could not find any place_id for the location')
    
else:
    raise SystemExit(f'An error occured fetching the forecast ({response.status_code}, {response.text})')

# output
forecast_summary

The Weather in Buenos Aires City is:


'Cloudy changing to possible rain in the afternoon. Temperature 11/14 °C.'

## Generate the "Update message"

Now that we've gathered all data for our update message, we can now generate a nice message using AI! We'll do this by sending a prompt to the OpenAI chat completions API to generate this message. But first let's add the data we retrieved from NewsAPI and MeteoSource to our prompt, so the OpenAI API can generate a relevant update for us. So, we will:

- Create a variable `system_message` with the system message
- Create a variable `user_message` with the user message
- Add the variables with the forecast and headlines we created before into the `user_message` variable

In [11]:
# Create the system_message variable
system_message = '''
You are an AI assistant tasked with generating a 'Morning Update' text that’s engaging and enjoyable for the user to listen to while having their morning coffee. The update should be about 2-5 minutes long, incorporating both a weather forecast and top 10 news headlines in a way that feels conversational, lively, and fits a specific tone (such as funny, serious, sarcastic, or motivational). Do not output anything else than the text, don't include any markup, lists, or other structural elements. The text will be sent to a text-to-speech API to generate an MP3, so make sure the output contains nothing that should not be read out loud.

Structure the monologue as follows:

1. Greeting: Start with a warm and welcoming greeting.
2. Weather Summary: Describe the day’s weather, infusing the chosen tone (e.g., funny, serious, etc.) to make it engaging.
3. News Headlines: Present each headline in the chosen tone, followed by a summary of the headline to give the listener a deeper insight into the headline.
4. Closing: Wrap up with a concluding remark that leaves the reader with a smile, positive thought, or playful nudge.

Be creative in how you incorporate the tone and style, ensuring that the text is engaging and enjoyable to listen to.
'''

# Create the user_message variable
user_message = f'''
Please generate a 'Morning Update' text in a funny and light tone.

Here is the Weather Forecast: 
```
{forecast_summary}
```

Here are the News Headlines in JSON format:
```
{headline_articles}
```

Generate the text as specified in the system prompt, following the structure of greeting, weather summary, 10 headlines, and a closing remark.
'''

When reading [the OpenAI chat completions documentation](https://platform.openai.com/docs/api-reference/chat/create), we notice that we need to use the `POST` HTTP verb. Previously, we only retrieved data from the API using the `GET` method. Now, we need to send data (the prompt) to the API to generate a new response. This is why we use the `POST` verb and the `post()` function from the `requests` library.

This API requires Bearer authentication. We need to add an `Authorization` header, similar to the MeteoSource API. The difference is that the value must be prefixed with the `Bearer` keyword, followed by the secret token or API key.

**Note:** OpenAI provides a user-friendly [Python package called `openai`](https://platform.openai.com/docs/libraries#python-library) that simplifies using the API. However, to understand the underlying concepts of APIs, we'll use the `requests` package and manually create the API requests.

- Create a dictionary called `openai_headers`
- Add the correct Bearer authorization header to the dictionary

In [12]:
openai_headers = {"authorization": f"Bearer {API_KEY_OPENAI}"}

Now on to the actual data we'll be sending to the chat completions API. Given we'll be sending a `POST` request, the data structure is a little more complex than the simple dictionary used for URL parameters.

For the **model** value we'll use the `gpt-4o-mini` model as it's the best model in terms of cost vs quality. 

For the **messages** value, we'll need to create a list, each entry is a dictionary with a `role` and `content` property. For our use-case we'll add 2 messages: one message with the `system` role and one with the `user` role, each corresponding to the messages we created before.

- Create a new dictionary called `completions_request_data`.
- Add a `model` item to the dictionary with our chosen language model
- Add a `messages` item to the dictionary with a list of messages. Each item in this list again is a dictionary with a `role` and `content` item.

In [13]:
completions_request_data = {
    "model": "gpt-4o-mini", 
    "messages": [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
}

Now we're ready to send the request to the OpenAI chat completions API. As mentioned before, we'll need to send a `POST` request, thus we won't be using the `get()` function anymore. `POST` requests also expect some data to be sent along with the message. When we're sending JSON formatted data, we can use the `json` argument, similar to how we add headers using the `headers` argument.

- Send the API request to the `/chat/completions` API.
- Evaluate if the response is successful.
- Check if the response contains a list of choices and inspect the structure of the choice object.
- Store the message content of the first choice in the `morning_update` variable.

In [14]:
response = requests.post("https://api.openai.com/v1/chat/completions", 
                        json=completions_request_data,
                        headers=openai_headers)

if (response.status_code == 200):
    response_json = response.json()
    
    if len(response_json['choices']) > 0:
        morning_update = response_json['choices'][0]['message']['content']
    else:
        raise SystemExit('Could not find any place_id for the location')
    
else:
    raise SystemExit(f'An error occured generating a message ({response.status_code}, {response.text})')
    

print(morning_update)

Good morning, sunshine! Grab your coffee, get cozy, and let’s dive into today’s delightful mix of fluff and stuff. 

First up, let’s chat about the weather. It’s kind of a moody day out there, with clouds acting like they just binge-watched a dramatic series. They’re sticking around for a bit, and then, you guessed it, they just might cry later today. So, dress accordingly—13 degrees Celsius, which is approximately “I should have worn a sweater but I thought it was spring” weather. Don’t say I didn’t warn you!

Now, let’s hop into the news headlines! 

Ananda Lewis, former MTV VJ who shared her breast cancer journey, has sadly passed away at 52. You know, she was that charismatic presence connecting music lovers with their favorite stars during the golden MTV years. It’s a bittersweet moment for all of us who grew up watching her shine.

Moving on, we’ve got some political drama—surprise! After a recent Trump speech, some soldiers were seen cheering and booing. Seems like military deco

## Text to Speech (TTS) to generate an audio file

With the message generated by OpenAI, we are now ready to create a spoken version of the text and save it as an MP3 for listening. OpenAI offers an excellent text-to-speech API that features a variety of natural-sounding voices for generating the audio file. We then start by reviewing the API documentation for the [OpenAI createSpeech Audio APIs](https://platform.openai.com/docs/api-reference/audio/createSpeech). 
We will:

- Create a new dictionary called `tts_request_data`
- Add items to the dictionary for all the required arguments

In [15]:
tts_request_data = {
    "model": "tts-1",
    "voice": "fable",
    "input": morning_update    
}

Now let's send the request, but this time we need to evaluate the response a bit differently.

This response will not contain readable text; instead, it will provide data for an MP3 audio file. We need to store the data returned by the API in a file on disk. Fortunately, Python includes the necessary file manipulation functions, such as `open()`, which allow us to do this. 

The `open()` function requires two arguments: the filename and the mode. The mode argument specifies the purpose for which the file is being opened. By passing the `wb` value to this argument, we can open the file for writing binary data. This is essential because audio files are binary files, and writing them in text mode could corrupt the data.

- Send the API request to the `/audio/speech` API.
- Evaluate if the response is successful.
- Write the response content to the `good_morning.mp3` file.
- Use the "_Show Workbook files_" option from the "_View_" menu to confirm the file was successfully generated.

In [16]:
response = requests.post("https://api.openai.com/v1/audio/speech", 
                        json=tts_request_data,
                        headers=openai_headers)

if (response.status_code == 200):
    file = open('good_morning.mp3', 'wb')
    file.write(response.content)
    file.close()
    print("Your audio file has been downloaded correctly in you hard drive. Enjoy it!")
   
else:
    raise SystemExit(f'An error occured generating the audio file ({response.status_code}, {response.text})')

Your audio file has been downloaded correctly in you hard drive. Enjoy it!


## Conclusion

In this project, we successfully demonstrated the powerful interaction between external APIs and OpenAI, showcasing the immense potential of integrating AI-driven services into practical applications. By leveraging the OpenAI API, we generated an audio file from text input, highlighting the seamless and efficient capabilities of modern AI technologies.

### Key Steps Undertaken:
1. **API Request**: We crafted a POST request to the OpenAI API, ensuring that the request payload and headers were correctly formatted.
2. **Response Handling**: We evaluated the response to ensure it was successful (HTTP status code 200). In the event of an error, we raised an appropriate exception with detailed information.
3. **File Writing**: Upon a successful response, we wrote the binary content of the response to an MP3 file, ensuring the file was properly closed after writing.
4. **Verification**: We recommended using the "_Show Workbook files_" option from the "_View_" menu to confirm the successful generation of the audio file.

### Achievements:
- **Integration with OpenAI API**: Demonstrated the ability to interact with the OpenAI API for text-to-speech conversion, emphasizing the power and flexibility of integrating external APIs with OpenAI.
- **Error Handling**: Implemented robust error handling to manage unsuccessful API responses, ensuring reliability and user trust.
- **File Management**: Ensured proper handling and writing of binary data to create an audio file, showcasing effective multimedia data management.

### Future Enhancements:
- **User Interface**: Develop a user-friendly interface to allow users to input text and download the generated audio file directly.
- **Extended Functionality**: Explore additional features such as different voice options, languages, and audio formats.
- **Automation**: Automate the process to handle batch requests and generate multiple audio files in one go.

This project is a significant addition to my Generative AI portfolio, illustrating the seamless integration of API services and the effective handling of multimedia data. The successful completion of this project underscores the transformative potential of combining external APIs with OpenAI, paving the way for creating dynamic and interactive audio content. The power of such integrations cannot be overstated, as they open up new avenues for innovation and efficiency in AI-driven applications.